# Tutorial: Analise de sentimentos de reviews com Gemini

Este notebook mostra como transformar feedbacks textuais em insights utilizando o modelo Gemini. Vamos configurar o ambiente, carregar um conjunto de reviews e automatizar a classificacao de sentimento antes de aprofundar nas causas de insatisfacao.


## Visao geral

Ao final voce sera capaz de:
- instalar e validar as dependencias essenciais;
- configurar com seguranca a chave de API do Gemini;
- carregar e inspecionar um conjunto de reviews em CSV;
- classificar sentimentos (Positiva, Negativa ou Neutra) de forma programatica;
- extrair razoes resumidas para feedbacks negativos.


## Pre-requisitos

- Python 3.10 ou superior.
- Acesso ao Google AI Studio com permissao para o modelo `gemini-2.5-flash`.
- Uma chave de API ativa armazenada em local seguro.
- Arquivo `reviews.csv` com uma coluna `reviewText` contendo os textos a serem avaliados.


## 1. Preparar o ambiente

Execute a celula abaixo para garantir que `google-genai` e `pandas` estejam instalados. Em plataformas gerenciadas (como Google Colab), confirme se voce tem permissao para instalar pacotes.


In [ ]:
%pip install --quiet --upgrade google-genai pandas

## 2. Configurar a chave de API

Armazene sua chave em uma variavel de ambiente antes de rodar o notebook. Substitua `COLE_SUA_CHAVE_AQUI` apenas se estiver testando localmente; em producao, prefira variaveis de ambiente ou cofres de segredos.


In [ ]:
import os

API_KEY = os.getenv('GOOGLE_API_KEY', 'COLE_SUA_CHAVE_AQUI')
if API_KEY == 'COLE_SUA_CHAVE_AQUI':
    raise ValueError("Defina a variavel de ambiente GOOGLE_API_KEY ou substitua 'COLE_SUA_CHAVE_AQUI' pelo valor real da sua chave.")

os.environ['GOOGLE_API_KEY'] = API_KEY
print('Chave configurada com sucesso.')

## 3. Inicializar o cliente Gemini

Com a chave carregada, criamos o cliente que enviara as requisicoes ao modelo. Mantemos o identificador do modelo em uma constante para facilitar futuras trocas.


In [ ]:
from google import genai

MODEL_ID = 'gemini-2.5-flash'
client = genai.Client()
print(f'Modelo pronto: {MODEL_ID}')

## 4. Carregar o conjunto de reviews

Atualize `CSV_PATH` conforme a localizacao do arquivo. O notebook espera encontrar a coluna `reviewText`. Caso seu dataset utilize outro nome, ajuste o codigo e a descricao abaixo.


In [ ]:
import pandas as pd

CSV_PATH = 'reviews.csv'  # ajuste para o caminho correto

try:
    df_reviews = pd.read_csv(CSV_PATH)
except FileNotFoundError as exc:
    raise FileNotFoundError(f'Arquivo nao encontrado em {CSV_PATH}. Atualize o caminho antes de prosseguir.') from exc

if 'reviewText' not in df_reviews.columns:
    raise KeyError("O arquivo deve conter uma coluna chamada 'reviewText'. Colunas encontradas: " f"{list(df_reviews.columns)}")

print(f'Total de reviews carregadas: {len(df_reviews)}')
df_reviews.head(3)

## 5. Explorar rapidamente os dados

Verifique a existencia de valores vazios e visualize alguns exemplos para garantir que os textos foram carregados corretamente.


In [ ]:
total_vazios = df_reviews['reviewText'].isna().sum()
print(f'Reviews sem texto: {total_vazios}')

exemplos = df_reviews['reviewText'].dropna().head(5)
for idx, texto in enumerate(exemplos, start=1):
    trecho = texto.replace('\n', ' ')[:120]
    print(f'{idx:02d}. {trecho}')

## 6. Definir o prompt e a funcao de analise de sentimento

Utilizaremos um prompt padronizado forcando a resposta em uma unica palavra (Positiva, Negativa ou Neutra). A funcao abaixo encapsula a chamada ao modelo e trata entradas vazias.


In [ ]:
from textwrap import dedent

SENTIMENT_PROMPT = dedent("""
Classifique o sentimento da resenha a seguir como UMA das opcoes: Positiva, Negativa ou Neutra.
Responda apenas com a palavra correspondente.

Resenha: {resenha}
""").strip()

def classificar_sentimento(texto: str) -> str:
    """Retorna Positiva, Negativa, Neutra ou Sem texto em caso de entrada vazia."""
    if not isinstance(texto, str) or not texto.strip():
        return 'Sem texto'

    prompt = SENTIMENT_PROMPT.format(resenha=texto.strip())

    try:
        resposta = client.models.generate_content(
            model=MODEL_ID,
            contents=prompt,
        )
    except Exception as exc:
        print(f'Falha ao analisar o texto: {exc}')
        return 'Erro'

    sentimento = resposta.text.strip().splitlines()[0]
    return sentimento

## 7. Classificar uma amostra de reviews

Para controlar custo e latencia, comecamos com uma amostra pequena. Ajuste o limite conforme sua cota de uso.


In [ ]:
AMOSTRA_LIMITE = 10  # altere conforme necessario

df_sample = df_reviews.head(AMOSTRA_LIMITE).copy()
resultados = []

for idx, texto in enumerate(df_sample['reviewText'], start=1):
    sentimento = classificar_sentimento(texto)
    resultados.append(sentimento)
    trecho = (texto or '').replace('\n', ' ')[:80]
    print(f'{idx:02d}. Sentimento: {sentimento} | Trecho: {trecho}')

df_sample['sentimento_gemini'] = resultados

colunas_saida = ['reviewText', 'sentimento_gemini']
df_sample[colunas_saida]

## 8. Investigar reviews negativas

Filtramos as resenhas marcadas como Negativa para entender pontos de atencao. Caso nao haja resultados, retorne ao passo anterior e aumente a amostra.


In [ ]:
negativas = df_sample[df_sample['sentimento_gemini'] == 'Negativa'].copy()

if negativas.empty:
    print('Nenhuma review negativa encontrada na amostra atual.')
else:
    print(f'Total de reviews negativas: {len(negativas)}')
    negativas[['reviewText', 'sentimento_gemini']]

## 9. Identificar motivos para insatisfacao

Criamos um segundo prompt para solicitar um resumo objetivo do motivo da avaliacao negativa. Limitamos a quatro chamadas para conter o uso inicial.


In [ ]:
MOTIVO_PROMPT = dedent("""
A resenha a seguir foi classificada como negativa. Em uma unica frase curta, explique o principal motivo da insatisfacao.
Foque no problema central citado pelo cliente e evite repetir o texto original.

Resenha: {resenha}
""").strip()

def explicar_motivo(texto: str) -> str:
    if not isinstance(texto, str) or not texto.strip():
        return 'Sem texto'

    prompt = MOTIVO_PROMPT.format(resenha=texto.strip())

    try:
        resposta = client.models.generate_content(
            model=MODEL_ID,
            contents=prompt,
        )
    except Exception as exc:
        print(f'Falha ao explicar o motivo: {exc}')
        return 'Erro'

    return resposta.text.strip()

if negativas.empty:
    print('Sem motivos a identificar porque nao ha reviews negativas.')
else:
    indices_alvo = negativas.head(4).index
    for idx in indices_alvo:
        texto = negativas.at[idx, 'reviewText']
        motivo = explicar_motivo(texto)
        negativas.at[idx, 'motivo_gemini'] = motivo
        print(f'Review {idx}: {motivo}')

    df_sample.loc[indices_alvo, 'motivo_gemini'] = negativas.loc[indices_alvo, 'motivo_gemini']
    negativas[['reviewText', 'motivo_gemini']]

## 10. Proximos passos

- Ampliar a amostra ou processar todo o dataset com controles de taxa (backoff, retries, cache).
- Persistir as classificacoes em um banco de dados ou planilha para acompanhamento historico.
- Treinar modelos locais com os rotulos gerados para reduzir custo em larga escala.
- Integrar a analise a dashboards (Looker, Power BI) ou notificacoes automaticas.

A partir daqui voce pode adaptar o prompt, mudar o modelo ou conectar o fluxo a interfaces maiores.
